<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/MESSAGE_SPAM_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/SMSSpamCollection.txt',sep='\t',names=['label','message'])

In [3]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [6]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [7]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
df.duplicated().sum()

403

In [9]:
df=df.drop_duplicates()

In [10]:
df.reset_index(drop=True,inplace=True)

In [11]:
df['label'].value_counts()

ham     4516
spam     653
Name: label, dtype: int64

In [12]:
df['message'][500]

"You should know now. So how's anthony. Are you bringing money. I've school fees to pay and rent and stuff like that. Thats why i need your help. A friend in need....|"

##TEXT PREPREOCESSING

In [13]:
import re
import nltk


In [14]:
stop_words=nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
from nltk.corpus import stopwords

In [16]:
from nltk.stem import PorterStemmer

In [17]:
ps=PorterStemmer()

In [18]:
corpus=[]
for i in range(len(df)):
  temp=re.sub('^a-zA-Z0-9',' ',df['message'][i])
  temp=temp.lower()
  temp=temp.split()
  temp=[ps.stem(k) for k in temp if k not in stopwords.words('english')]
  temp=' '.join(temp)
  corpus.append(temp)


###using bag of words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
cv=CountVectorizer(max_features=2500,binary=True) #only the top 2500 most frequent words will be considered.

In [21]:
X=cv.fit_transform(corpus).toarray()

In [22]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
y=df['label'].apply(lambda x: 0 if x=='ham' else 1)

In [24]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5164    1
5165    0
5166    0
5167    0
5168    0
Name: label, Length: 5169, dtype: int64

In [25]:
type(y)

pandas.core.series.Series

In [26]:
y=y.values

In [27]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=43)

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
rf=RandomForestClassifier()

In [32]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [33]:
y_pred=rf.predict(X_test)


In [34]:
from sklearn.metrics import accuracy_score,classification_report

In [35]:
accuracy_score(y_test,y_pred)

0.9796905222437138

In [36]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       890
           1       0.98      0.88      0.92       144

    accuracy                           0.98      1034
   macro avg       0.98      0.94      0.96      1034
weighted avg       0.98      0.98      0.98      1034



In [37]:
# creating tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
tfidf=TfidfVectorizer(max_features=2500)

In [39]:
X=tfidf.fit_transform(corpus).toarray()

In [40]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [42]:
model2=RandomForestClassifier()

In [43]:
model2.fit(X_train,y_train)

RandomForestClassifier()

In [44]:
y_pred=model2.predict(X_test)

In [45]:
accuracy_score(y_test,y_pred)

0.9816247582205029

##WORD2VEC

In [71]:
!pip install gensim

In [72]:
import gensim

In [73]:
from gensim.models import Word2Vec,KeyedVectors

In [74]:
import gensim.downloader as api

In [75]:
wv=api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [80]:
embedding=[]
for i in corpus:
  words=nltk.word_tokenize(i)
  embedded_words=[wv[word] for word in words if word in wv.vocab]
  sent=' '.join(embedded_words)
  embedding.append(sent)



AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [79]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for token in tokens:
        if token in pretrained_embeddings:
            embeddings.append(pretrained_embeddings[token])

In [46]:
corpus

['go jurong point, crazy.. avail bugi n great world la e buffet... cine got amor wat...',
 'ok lar... joke wif u oni...',
 "free entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question(std txt rate)t&c' appli 08452810075over18'",
 'u dun say earli hor... u c alreadi say...',
 'nah think goe usf, live around though',
 "freemsg hey darl 3 week' word back! i'd like fun still? tb ok! xxx std chg send, £1.50 rcv",
 'even brother like speak me. treat like aid patent.',
 "per request 'mell mell (oru minnaminungint nurungu vettam)' set callertun callers. press *9 copi friend callertun",
 'winner!! valu network custom select receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'mobil 11 month more? u r entitl updat latest colour mobil camera free! call mobil updat co free 08002986030',
 "i'm gonna home soon want talk stuff anymor tonight, k? i'v cri enough today.",
 'six chanc win cash! 100 20,000 pound txt> csh11 send 87

In [48]:
from gensim.utils import simple_preprocess

In [51]:
words=[]
for sentence in corpus:
  sent_token=nltk.sent_tokenize(sentence)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

 simple_preprocess()---This function typically converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long.


In [50]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [52]:
words ## we get all the unique words present in each sentences

[['go',
  'jurong',
  'point',
  'crazy',
  'avail',
  'bugi',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amor',
  'wat'],
 ['ok', 'lar', 'joke', 'wif', 'oni'],
 ['free',
  'entri',
  'wkli',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkt',
  'st',
  'may',
  'text',
  'fa',
  'receiv',
  'entri',
  'question',
  'std',
  'txt',
  'rate',
  'appli',
  'over'],
 ['dun', 'say', 'earli', 'hor', 'alreadi', 'say'],
 ['nah', 'think', 'goe', 'usf', 'live', 'around', 'though'],
 ['freemsg', 'hey', 'darl', 'week', 'word', 'back'],
 ['like', 'fun', 'still'],
 ['tb', 'ok'],
 ['xxx', 'std', 'chg', 'send', 'rcv'],
 ['even', 'brother', 'like', 'speak', 'me'],
 ['treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'mell',
  'mell',
  'oru',
  'minnaminungint',
  'nurungu',
  'vettam',
  'set',
  'callertun',
  'callers'],
 ['press', 'copi', 'friend', 'callertun'],
 ['winner'],
 ['valu', 'network', 'custom', 'select', 'receivea', 'prize', 'reward'],
 ['claim', 'call',

###NOW LETS TRAIN OUR WORD2VEC MODEL FROM SCRATCH

In [53]:
import gensim

In [54]:
model=gensim.models.Word2Vec(words,window=5,min_count=2)

window: This parameter specifies the maximum distance between the current and predicted word within a sentence. In other words, it defines the context window size for the model. Words outside this window are not considered as context words during training.

In [55]:
model.wv.index_to_key

['call',
 'go',
 'get',
 'ur',
 'gt',
 'lt',
 'come',
 'ok',
 'know',
 'free',
 'like',
 'want',
 'day',
 'got',
 'time',
 'good',
 'love',
 'you',
 'now',
 'll',
 'text',
 'send',
 'need',
 'it',
 'me',
 'one',
 'see',
 'txt',
 'today',
 'think',
 'lor',
 'home',
 'stop',
 'take',
 'that',
 'still',
 'back',
 'tell',
 'da',
 'make',
 'dont',
 'hi',
 'say',
 'repli',
 'phone',
 'new',
 'ask',
 'mobil',
 'pleas',
 'hope',
 'week',
 'co',
 'miss',
 'work',
 'night',
 'give',
 'wait',
 'well',
 'meet',
 'oh',
 'much',
 'hey',
 'msg',
 'wat',
 'great',
 'tri',
 'no',
 'thing',
 'dear',
 'min',
 'number',
 'later',
 'claim',
 'way',
 'thank',
 'let',
 'there',
 'pl',
 'happi',
 'yeah',
 'sorri',
 'im',
 'prize',
 'babe',
 'www',
 'feel',
 'can',
 'right',
 'tomorrow',
 'friend',
 'find',
 'cash',
 'realli',
 'pick',
 'said',
 'even',
 'sure',
 'keep',
 'would',
 'lol',
 'last',
 'care',
 'how',
 'also',
 'leav',
 'life',
 'buy',
 'look',
 'contact',
 'sleep',
 'win',
 'finish',
 'messag',
 

In [57]:
model.corpus_count   #total no. corpus count

9808

In [58]:
model.epochs

5

In [64]:
model.wv.similar_by_word('happy')

[('right', 0.988314688205719),
 ('may', 0.9882320761680603),
 ('said', 0.9881101250648499),
 ('ask', 0.9880995750427246),
 ('home', 0.9880809187889099),
 ('im', 0.9880728721618652),
 ('entri', 0.9880266785621643),
 ('babe', 0.9880256056785583),
 ('sure', 0.988024890422821),
 ('cash', 0.9879879355430603)]

In [66]:
def avg_word2vec(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [68]:
X=[]
for i in range(len(words)):
  X.append(avg_word2vec(words[i]))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [69]:
type(X)

list

In [87]:
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Ensure that nan values are replaced with 0.0
X = [np.nan_to_num(sequence, nan=0.0) for sequence in X]

# Ensure that each sequence is a list containing one element (each element is wrapped inside a list)
X = [[val] if not isinstance(val, (list, tuple)) else val for val in X]

# Pad sequences to ensure uniform length, with each sequence having a length of 100
X_padded = pad_sequences(X, maxlen=100, dtype='float32', padding='post', truncating='post', value=0.0)

# Reshape sequences to have a shape of (1, 1, 1, 100)
X_new = np.expand_dims(X_padded, axis=(0, 1, 2))

# Optionally, convert X_new to a NumPy array
X_new = np.array(X_new)

ValueError: Shape of sample (1, 1, 1, 1) of sequence at position 31 is different from expected shape (1, 1, 1, 1, 100)

In [75]:
X[0].shape

(100,)

In [76]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [9808, 5169]

In [77]:
y.shape

(5169,)

In [80]:
X_new=np.array(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9808,) + inhomogeneous part.